## Import libraries and dependencies

In [13]:
import logging
import sys
import time
import csv
import cv2
import os
from json import dumps, loads
from time import sleep
from kafka import KafkaProducer

## Videos preparation

In [2]:
# Video Generation
def generate_video(image_folder):
    # image_folder = '/home/cthi/UIT/IE212/UAV-benchmark-M/M0101'
    folder_name = image_folder.split('/')[-1]
    video_path = '../../videos/' + folder_name + '.mp4'

    print(video_path)

    # Listed images
    images = [img for img in os.listdir(image_folder)] 
    images = sorted(images)
    height = 540
    width = 1024

    # fourcc = cv2.VideoWriter_fourcc(*'XVID')  # XVID codec
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # XVID codec
    video = cv2.VideoWriter(video_path, fourcc, 30, (width, height))

    # Appending the images to the video one by one 
    for image in images:  
        video.write(cv2.imread(os.path.join(image_folder, image)))  

    video.release()

    return video_path

In [15]:
def publish_camera(topic, imgs):
    video_path = generate_video(imgs)

    # Create producer
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    video = cv2.VideoCapture(video_path)
    
    # Send frames of generated video
    try:
        print(f'Publishing {topic}')
        k = 1
        
        while(True):    
            __, frame = video.read()
            __, buffer = cv2.imencode('.jpg', frame)
            producer.send(topic, key=k.to_bytes(4, byteorder='big'), value=buffer.tobytes())
            k+=1

    except RuntimeError:
        producer.flush()
        print("\nExiting...")
        sys.exit(1)

## Producers

In [ ]:
topic_2 = 'drone_2'
image_folder_2 = '/home/cthi/UIT/IE212/UAV-benchmark-M/M0201'

publish_camera(topic_2, image_folder_2)

../../videos/M0201.mp4


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.conn:<BrokerConnection node_id=0 host=cthi-Inspiron-5515:9092 <connecting> [IPv4 ('127.0.1.1', 9092)]>: connecting to cthi-Inspiron-5515:9092 [('127.0.1.1', 9092) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=0 host=cthi-Inspiron-5515:9092 <connecting> [IPv4 ('127.0.1.1', 9092)]>: Connection complete.

Publishing drone_2


error: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:1134: error: (-215:Assertion failed) !image.empty() in function 'imencode'
